In [1]:
import pandas as pd
from matplotlib.colors import ListedColormap
from faerun import Faerun
from rdkit import Chem 
from rdkit.Chem import Lipinski, Descriptors, rdMolDescriptors, AllChem, PandasTools, rdchem
from pandarallel import pandarallel
import numpy as np
pandarallel.initialize(progress_bar=False)
import tmap as tm
from map4 import MAP4Calculator
import os
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
from rdkit.Chem.AtomPairs import Pairs
folder = "/data/coconut/"

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
bacteria_names = ["bacillus", "bacta", "bacteria"]
fungi_names = ["aspergillus"]
def simple_tax(textTaxa):
    Taxa_clean = []
    textTaxa = textTaxa.split(",")
    for tax in textTaxa:
        tax = tax.strip().replace("[", "").replace("]", "").replace(" ","").replace("\t","")
        #print(tax)
        Taxa_clean.append(tax)
    
    if "Homosapiens" in Taxa_clean:
        #print("found Homo sapiens")
        simple_tax = "Homo_sapiens"
    elif "animal" in Taxa_clean or "animals" in Taxa_clean:
        simple_tax = "animal"
    elif "bacteria" in Taxa_clean:
        simple_tax = "bacteria"
    elif "fungi" in Taxa_clean:
        simple_tax = "fungi"
    elif "plants" in Taxa_clean or "plant" in Taxa_clean:
        simple_tax = "plants"
    elif "marine" in Taxa_clean:
        simple_tax = "marine"
    else:
        simple_tax = "other"
        for tax in Taxa_clean:
            for bacteria_name in bacteria_names:
                if bacteria_name in tax.lower():
                    simple_tax = "bacteria"
                    break
        for tax in Taxa_clean:
            for fungi_name in fungi_names:
                if fungi_name in tax.lower():
                    simple_tax = "fungi"
                    break
    return simple_tax 
    
def clean_source(citationDOI):
    if len(citationDOI) >=10:
        return 1
    elif len(citationDOI) >=3:
        return -1
    else:
        return 0

In [17]:
def canonical_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        smiles = Chem.MolToSmiles(mol)
        return smiles
    else:
        return np.nan
    
MAP4 = MAP4Calculator(dimensions=1024)
def calc_map4(smiles):
    mol = Chem.MolFromSmiles(smiles)
    smiles = Chem.MolToSmiles(mol, isomericSmiles=False)
    mol = Chem.MolFromSmiles(smiles)
    map4 = MAP4.calculate(mol)
    return np.array(map4)

def calc_hba(smiles):
    mol = Chem.MolFromSmiles(smiles)
    hba = Lipinski.NumHAcceptors(mol)
    return hba

def calc_mw(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mw = Descriptors.ExactMolWt(mol)
    return mw

def calc_hbd(smiles):
    mol = Chem.MolFromSmiles(smiles)
    hbd = Lipinski.NumHDonors(mol)
    return hbd

def calc_fcsp3(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fcsp3 = Lipinski.FractionCSP3(mol)
    return fcsp3

def calc_logp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    logp = Descriptors.MolLogP(mol)
    return logp

def is_lipinski(row):
    rules = 0
    if row.MW >= 500:
        rules += 1
    if row.HBA > 10:
        rules += 1
    if row.HBD > 5:
        rules += 1
    if row.aLogP > 5:
        rules += 1
    if rules > 1 :
        return 0
    else:
        return 1

def create_label(row):
    smiles = row.SMILES
    ID = row.coconut_id.strip()
    link = f'https://coconut.naturalproducts.net/compound/coconut_id/{ID}'
    label = f'{smiles}__<a target="_blank" href={link}>{ID}</a>'
    #label = smiles+"__"+NPAID+"__"+link
    return label

def has_aminoacid(smiles):
    m = Chem.MolFromSmiles(smiles)
    # generic dipeptide (does not hit Pro)
    if m.HasSubstructMatch(Chem.MolFromSmarts('[NX3,NX4+][CH1,CH2][CX3](=[OX1])[NX3,NX4+][CH1,CH2][CX3](=[OX1])[O,N]')):
        return 1
    else:
        return 0
    
def has_sugar(smiles):
    m = Chem.MolFromSmiles(smiles)
    # acetal moiety
    if m.HasSubstructMatch(Chem.MolFromSmarts('[CR][OR][CHR]([OR0,NR0])[CR]')):
        return 1
    else:
        return 0
    
def calc_ecfp4(smiles):
    m = Chem.MolFromSmiles(smiles)
    return AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=1024)

def calc_ap(smiles, l=1024):
    m = Chem.MolFromSmiles(smiles)
    sparse = Pairs.GetAtomPairFingerprintAsBitVect(m).GetOnBits()
    ap = np.zeros(l)
    for i in sparse:
        ap[i%l] = 1
    return ap


origins = ["plants", "fungi", "bacteria", "animal", "Homo_sapiens"]
def origin(simple_tax):
    if simple_tax in origins:
        return int(origins.index(simple_tax))
    else:
        return 5

## DB extraction

In [4]:
if not os.path.exists(folder+"coconut_futher_analysis.pkl"):
    sdfFile = os.path.join("data/COCONUT_DB.sdf")
    coconut_ = PandasTools.LoadSDF(sdfFile)
    print(len(coconut_))

    coconut_ = coconut_.query("textTaxa != '[notax]'").copy()
    print(len(coconut_))

    coconut_["canonical_smiles"] = coconut_.SMILES.parallel_map(canonical_smiles)
    coconut_ = coconut_.dropna().copy()
    print(len(coconut_))

    coconut_["has_source"] = coconut_.citationDOI.map(clean_source)
    #coconut = coconut.query("has_source == 1").copy()
    print(len(coconut_))
    pd.to_pickle(coconut_, folder + "coconut_futher_analysis.pkl")
else:
    coconut_ = pd.read_pickle(folder + "coconut_futher_analysis.pkl")

In [9]:
coconut_["canonical_smiles"] = coconut_.SMILES.parallel_map(canonical_smiles)
coconut_ = coconut_.dropna().copy()

RDKit ERROR: [12:45:03] Explicit valence for atom # 34 N, 5, is greater than permitted
RDKit ERROR: [12:45:03] Explicit valence for atom # 65 N, 5, is greater than permitted
RDKit ERROR: [12:45:04] Explicit valence for atom # 25 N, 5, is greater than permitted
RDKit ERROR: [12:45:04] Explicit valence for atom # 37 N, 5, is greater than permitted
RDKit ERROR: [12:45:04] Explicit valence for atom # 36 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for atom # 16 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for atom # 8 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for atom # 8 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for atom # 44 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for atom # 33 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for atom # 39 N, 5, is greater than permitted
RDKit ERROR: [12:45:05] Explicit valence for 

In [10]:
coconut_extra = coconut_.query("has_source == 0 or has_source == -1").copy()
coconut = coconut_.query("has_source == 1").copy()

400837
135091
135091
135091

In [11]:
print(len(coconut))

67730


In [24]:
print(len(coconut_))

135052


In [12]:
print(len(coconut_extra))

67322


## Properties calculation

In [21]:
if not os.path.exists(folder+"coconut_futher_analysis_prop.pkl"):
    coconut_extra["simple_tax"] = coconut_extra.textTaxa.map(simple_tax)
    coconut_extra["MAP4"] = coconut_extra.canonical_smiles.parallel_map(calc_map4)
    coconut_extra["fcsp3"] = coconut_extra.canonical_smiles.parallel_map(calc_fcsp3)
    coconut_extra["HBA"] = coconut_extra.canonical_smiles.parallel_map(calc_hba)
    coconut_extra["HBD"] = coconut_extra.canonical_smiles.parallel_map(calc_hbd)
    coconut_extra["aLogP"] = coconut_extra.canonical_smiles.parallel_map(calc_logp)
    coconut_extra["MW"] = coconut_extra.canonical_smiles.parallel_map(calc_mw)
    coconut_extra["isLipinski"] = coconut_extra.apply(is_lipinski, axis=1)
    coconut_extra["isPeptide"] = coconut_extra.canonical_smiles.parallel_map(has_aminoacid)
    coconut_extra["hasSugar"] = coconut_extra.canonical_smiles.parallel_map(has_sugar)
    coconut_extra["TMAPlabel"] = coconut_extra.apply(create_label, axis=1)
    coconut_extra["origin"] = coconut_extra.simple_tax.map(origin)
    coconut_extra["ecfp4"] = coconut_extra.canonical_smiles.parallel_map(calc_ecfp4)
    coconut_extra["ap"] = coconut_extra.canonical_smiles.parallel_map(calc_ap)
    pd.to_pickle(coconut_extra, folder + "coconut_futher_analysis_prop.pkl")
else:
    coconut_extra = pd.read_pickle(folder + "coconut_futher_analysis_prop.pkl")

In [16]:
coconut_extra.simple_tax

124         plants
2254        plants
2307        plants
2770      bacteria
3325      bacteria
            ...   
401608      plants
401609    bacteria
401617      plants
401621      plants
401623      plants
Name: simple_tax, Length: 67322, dtype: object

In [19]:
if not os.path.exists(folder+"coconut_prop.pkl"):    
    coconut["simple_tax"] = coconut.textTaxa.map(simple_tax)
    coconut["MAP4"] = coconut.canonical_smiles.parallel_map(calc_map4)
    coconut["fcsp3"] = coconut.canonical_smiles.parallel_map(calc_fcsp3)
    coconut["HBA"] = coconut.canonical_smiles.parallel_map(calc_hba)
    coconut["HBD"] = coconut.canonical_smiles.parallel_map(calc_hbd)
    coconut["aLogP"] = coconut.canonical_smiles.parallel_map(calc_logp)
    coconut["MW"] = coconut.canonical_smiles.parallel_map(calc_mw)
    coconut["isLipinski"] = coconut.apply(is_lipinski, axis=1)
    coconut["isPeptide"] = coconut.canonical_smiles.parallel_map(has_aminoacid)
    coconut["hasSugar"] = coconut.canonical_smiles.parallel_map(has_sugar)
    coconut["TMAPlabel"] = coconut.apply(create_label, axis=1)
    coconut["origin"] = coconut.simple_tax.map(origin)
    coconut["ecfp4"] = coconut.canonical_smiles.parallel_map(calc_ecfp4)
    coconut["ap"] = coconut.canonical_smiles.parallel_map(calc_ap)
    pd.to_pickle(coconut, folder + "coconut_prop.pkl")
else:
    coconut = pd.read_pickle(folder + "coconut_prop.pkl")

In [20]:
coconut.columns

Index(['coconut_id', 'inchi', 'inchikey', 'SMILES', 'sugar_free_smiles',
       'molecular_formula', 'molecular_weight', 'citationDOI', 'textTaxa',
       'name', 'synonyms', 'NPL_score', 'number_of_carbons',
       'number_of_nitrogens', 'number_of_oxygens', 'number_of_rings',
       'total_atom_number', 'bond_count', 'found_in_databases',
       'murko_framework', 'alogp', 'apol', 'topoPSA', 'ID', 'ROMol',
       'has_source', 'canonical_smiles', 'simple_tax', 'MAP4', 'fcsp3', 'HBA',
       'HBD', 'aLogP', 'MW', 'isLipinski', 'isPeptide', 'hasSugar',
       'TMAPlabel', 'origin', 'has_lipo_chain', 'has_ring', 'has_sterol',
       'n_rings_nosugar', 'mw_nolipidtail_nosugar', 'smarts', 'is_terminal',
       'ecfp4', 'ap'],
      dtype='object')